In [1]:
import pandas as pd

In [2]:
moex = 'https://www.moex.com/'
init = 'a20'  # 4 квартал 2011 г.
urls = (('a685', 'a1127', 'a1244', 'a1465'), ('a1688', 'a1823', 'a2045', 'a2261'),
        ('a2474', 'a2702', 'a2813', 'a2987'), ('a3048', 'a3177', 'a3369', 'a3503'),
        ('a3601', 'a3691', 'a3794', 'a3882'), ('a4027', 'a4094', 'a4184', 'a4258'),
        ('a4318', 'a4377', 'a4420', 'a6401'), ('a6861', 'a6997', 'a7080', 'a7150'),
        ('a7246', 'a7305', 'a7396', 'a7605'))  # квартальные за 2012-2020 гг.

In [3]:
years = range(2012, 2021)
quarters = range(1, 5)

In [4]:
def parser(url):
    df = pd.read_html(url, index_col=0, thousands=' ', decimal=',')[0]
    df.columns = ['name', 'category', 'id', 'volume', 'price', 'cap']
    df.index.name = 'code'
    return df

In [5]:
def string_handler(string):
    if type(string) == str:
        x = bytes(string, 'latin1')
        x = x.replace(b'\xa0', b'')
        x = x.replace(b',', b'.')
        x = x.decode('latin1')
        x = x.replace(' ', '', 10)
        return x
    return string

In [6]:
df = parser(moex + init)

for i, year in enumerate(years):
    for j, quarter in enumerate(quarters):
        df = df.merge(parser(moex + urls[i][j]), how='outer',
                      left_index=True, right_index=True,
                      suffixes=(None, '_' + str(year) + '_' + str(quarter)))

In [7]:
for col in df.columns:
    if 'cap' in col:
        df[col] = df[col].apply(string_handler).astype(float)
    elif 'volume' in col:
        df[col] = pd.to_numeric(df[col].apply(string_handler), errors='coerce')

In [8]:
true_values = dict.fromkeys(df.index.values, [0, 1, 2])
for col in df.columns:
    if 'name' in col:
        for idx in true_values.keys():
            if type(df[col].loc[idx]) == str:
                true_values[idx][0] = df[col].loc[idx]
    elif 'category' in col:
        for idx in true_values.keys():
            if type(df[col].loc[idx]) == str:
                true_values[idx][1] = df[col].loc[idx]
    elif 'id' in col:
        for idx in true_values.keys():
            if type(df[col].loc[idx]) == str:
                true_values[idx][2] = df[col].loc[idx]

In [9]:
df['name'] = df.merge(pd.DataFrame(true_values,
                                   columns=['true_names', 'true_categories', 'true_id']),
                      left_index=True, right_index=True, how='inner')['true_names']
df['category'] = df.merge(pd.DataFrame(true_values,
                                   columns=['true_names', 'true_categories', 'true_id']),
                      left_index=True, right_index=True, how='inner')['true_categories']
df['id'] = df.merge(pd.DataFrame(true_values,
                                   columns=['true_names', 'true_categories', 'true_id']),
                      left_index=True, right_index=True, how='inner')['true_id']

In [11]:
for col in df.columns:
    if ('name' in col or 'category' in col or 'id' in col)\
    and (col != 'name' and col != 'category' and col != 'id'):
        del df[col]

In [14]:
true_values = dict.fromkeys(df.index.values, [0, 1, 2])

In [17]:
true_values['ABBN'][0] = 5

In [20]:
[[] * 10]

[[]]